# Imports

In [ ]:
import numpy as np
import tensorboard as tb
from tensorboardX import SummaryWriter

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'
matplotlib.rcParams['figure.figsize'] = (15, 5)

In [ ]:
%run ../utils/files.py

In [ ]:
# TB_ID = 'eIkFKGjGSZmJjo5DmDomkA'
TB_ID = 'VXpHYn3eTYCOtZC9evpZsQ'

# Load data

In [ ]:
experiment = tb.data.experimental.ExperimentFromDev(TB_ID)

In [ ]:
df = experiment.get_scalars()
df.head()

In [ ]:
runs = list(df['run'].unique())
len(df), len(runs)

# Add chex-f1

In [ ]:
for run in runs:
    run_id = RunId(run, debug=False, task='rg', experiment='best-cnn')
    writer = SummaryWriter(get_tb_log_folder(run_id))

    d = df.loc[df['run'] == run]
    d = d.loc[d['tag'].str.contains('chex_f1')]
    d = d.loc[~d['tag'].str.contains('No_Finding')]
    d2 = d.replace(r'chex_f1_\w+/(\w+)', r'\1', regex=True)
    s = d2.groupby(['tag', 'step'])['value'].apply(np.mean)

    for split in ('train', 'val'):
        for step, value in s[split].items():
            writer.add_scalar(f'Chex_f1_woNF/{split}', value, step)
            
    writer.close()

In [ ]:
run

In [ ]:
for step, value in curve.items():
    continue

In [ ]:
s

In [ ]:
n_rows = 2
n_cols = 1

print(run)
for i, split in enumerate(('train', 'val')):
    plt.subplot(n_rows, n_cols, i + 1)
    plt.title(split)
    plt.plot(s[split].index, s[split].values)

# Re-write metrics

Fix names

In [ ]:
def _rename_tag(tag):
    return tag.replace('wonf', 'woNF')

In [ ]:
_rename_tag('chexf1')

In [ ]:
runs = [
    '0420_223832_iu-x-ray_lstm-v2_lr0.0001_precnn-0420-174100_normS_size256_front',
    '0421_062547_iu-x-ray_lstm-att-v2_lr0.0001_precnn-0420-174100_normS_size256_front',
]

In [ ]:
for run in runs:
    # new_run_name = f'{run}__copy'
    run_id = RunId(run, debug=False, task='rg', experiment='best-cnn-fixed')
    writer = SummaryWriter(get_tb_log_folder(run_id))

    sub_df = df.loc[df['run'] == run]
    for index, row in sub_df.iterrows():
        tag = row['tag']
        step = row['step']
        value = row['value']

        tag = _rename_tag(tag)
        writer.add_scalar(tag, value, step)
    
    writer.close()